In [40]:
#import sys
#!{sys.executable} -m pip install keras==2.4.0
import tensorflow
import keras
import wandb
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation, Conv1D, Flatten, MaxPooling1D
from tensorflow.python.keras import regularizers

In [11]:
wandb.init()
config = wandb.config

# Set parameters
config.vocab_size = 1000
config.maxlen = 1000
config.batch_size = 32
config.embedding_dims = 10
config.filters = 16
config.kernel_size = 3
config.hidden_dims = 250
config.epochs = 10

In [25]:
df = pd.read_csv('processed_data/clean_data.csv')
df.dropna()
delete_row = df[df['Years'] == 0].index
df = df.drop(delete_row)
df['Years'].unique()

array([2018, 2017, 2019])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(df['Tweets with no Stopwords'], df['Years'], test_size = 0.33,
                                                   random_state = 2)

In [27]:
tokenizer = text.Tokenizer(num_words = config.vocab_size)
tokenizer.fit_on_texts(X_train.astype(str))
X_train_tokenized = tokenizer.texts_to_matrix(X_train.astype(str))
X_test_tokenized = tokenizer.texts_to_matrix(X_test.astype(str))
X_train_seq = sequence.pad_sequences(X_train_tokenized, maxlen = config.maxlen)
X_test_seq = sequence.pad_sequences(X_test_tokenized, maxlen = config.maxlen)

In [28]:
# One-hot encoding
y_train_encoded = LabelBinarizer().fit_transform(y_train)
y_test_encoded = LabelBinarizer().fit_transform(y_test)

In [36]:
label_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train.ravel())
print(label_weights)
label_weights = {i:label_weights[i] for i in range(len(label_weights))} # Create dictionary
print(label_weights)

[1.21118492 0.5432297  2.99831792]
{0: 1.21118492304933, 1: 0.5432297008596592, 2: 2.9983179213948445}


/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[2017 2018 2019], y=[2017 2018 2018 ... 2017 2017 2018] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [34]:
# Check shapes
print(X_train_seq.shape)
print(X_test_seq.shape)
print(y_train_encoded.shape)
print(y_test_encoded.shape)

(702308, 1000)
(345913, 1000)
(702308, 3)
(345913, 3)


In [41]:
def define_cnn_model(dropout_rate, l1_value, l2_value):
    model = Sequential()
    model.add(Embedding(config.vocab_size, config.embedding_dims, input_length = config.maxlen))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(config.filters, config.kernel_size, 
                     kernel_regularizer = regularizers.l1_l2(l1 = l1_value, l2 = l2_value),
                     padding = 'valid', activation = 'relu'))
    model.add(MaxPooling1D())
    model.add(Conv1D(config.filters, config.kernel_size, padding = 'valid', activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(config.hidden_dims, activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
# Define the drop out grid
dropout_grid = [0.1, 0.5, 0.9]
l1_grid = [2**-5, 2**-6, 2**-7, 2**-8]
l2_grid = [2**-5, 2**-6, 2**-7, 2**-8]
tot = len(dropout_grid) * len(l1_grid) * len(l2_grid)

# Variables for the best result
scores = []
best_history = [] # place holder
best_ind = 0
best_acc = 0

# Loop through each combination
pos = 0
for ii in dropout_grid:
    for jj in l1_grid:
        for kk in l2_grid:
            pos = pos + 1
            print("Fitting the ", pos, "/", tot , " model")
            # define the model
            curr_model = define_cnn_model(ii, jj, kk)
            
            # train the model
            curr_history = curr_model.fit(X_train_seq, y_train_encoded, epochs = 10, class_weight = label_weights,
                                          batch_size = 64, validation_data = (X_test_seq, y_test_encoded), verbose = 1)
            curr_acc = st.mean(curr_history.history['val_acc'][5:10])
            
            # save the best result
            if best_acc < curr_acc:
                best_acc = curr_acc
                best_ind = pos - 1
                best_history = curr_history
                
print(best_acc)
print(best_ind)

Fitting the  1 / 48  model
Train on 702308 samples, validate on 345913 samples
Epoch 1/10
199104/702308 [=======>......................] - ETA: 11:32 - loss: 0.7315 - acc: 0.6244